## Punto 12 con funzione

## <span style='color:red'>12) For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.</span>

In [1]:
import pandas as pd
import datetime 
import numpy as np

In [2]:
# scrivo directory
dire = '/Users/alessandropontini/Desktop/progetto_python/additional-kiva-snapshot/loans_lenders.csv'
dire1 = '/Users/alessandropontini/Desktop/progetto_python/additional-kiva-snapshot/loans.csv'
dire2 = '/Users/alessandropontini/Desktop/progetto_python/additional-kiva-snapshot/lenders.csv'
dire3 = '/Users/alessandropontini/Desktop/progetto_python/additional-kiva-snapshot/country_stats.csv'

In [3]:
# df temporanei

temp_df3 = pd.DataFrame()
temp_df4 = pd.DataFrame()
temp_df_NA_lenders = pd.DataFrame()
total_lenders_amount = pd.DataFrame()

In [4]:
# leggo il file loans, ma per la quantità di dati che devo leggere preferisco una lettura più accurata
# successivamente preferisco leggere le date come tali e non come stringhe.
df_loans = pd.read_csv(dire1, parse_dates=['disburse_time', 'planned_expiration_time'])

In [5]:
temp_df = df_loans.loc[df_loans['disburse_time'] > df_loans['planned_expiration_time']]

In [6]:
x = temp_df.loc[:,('disburse_time')]
temp_df.loc[:,('disburse_time')] = temp_df.loc[:,('planned_expiration_time')]
temp_df.loc[:,('planned_expiration_time')] = x

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [7]:
# uso funzione isin per vedere i loan_id che coincidano e cambio df_loan con i nuovi di df_temp corretti
df_loans.loc[df_loans.loan_id.isin(temp_df.loan_id), ['disburse_time', 'planned_expiration_time']] = temp_df[['disburse_time', 'planned_expiration_time']]

In [8]:
# creo una colonna per la duration e faccio differenza, dopo di che mostro il risultato
df_loans['duration'] = df_loans['planned_expiration_time'] - df_loans['disburse_time']
df_loans[['loan_id','duration']]

,loan_id,duration
0,657307,53 days 19:30:06
1,657259,96 days 14:25:07
2,658010,37 days 13:10:05
3,659347,34 days 19:10:02
4,656933,57 days 22:10:02
...,...,...
1419602,988180,39 days 17:00:03
1419603,988213,39 days 08:40:07
1419604,989109,51 days 14:20:04
1419605,989143,63 days 00:50:02


Sistemato il codice

In [9]:
df_loans.duration[df_loans.duration < '0']

Series([], Name: duration, dtype: timedelta64[ns])

## Tolgo tutti i null per non aver problemi successivamente

In [10]:
df_loans.head(5)

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model,duration
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,...,7.0,3,2,1,NaN,female,true,irregular,field_partner,53 days 19:30:06
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,...,8.0,11,2,1,NaN,female,true,monthly,field_partner,96 days 14:25:07
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,...,14.0,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner,37 days 13:10:05
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,...,14.0,21,2,1,user_favorite,female,true,monthly,field_partner,34 days 19:10:02
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,...,7.0,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner,57 days 22:10:02


In [11]:
temp_df = df_loans.dropna()

In [12]:
temp_df.loc[:,('disburse_time')]=temp_df.loc[:,('disburse_time')].dt.tz_localize(None)

In [13]:
temp_df.loc[:,('planned_expiration_time')]=temp_df.loc[:,('planned_expiration_time')].dt.tz_localize(None)

In [15]:
temp_df.loc[:,('differenza')] = temp_df.loc[:,('planned_expiration_time')].dt.year - temp_df.loc[:,('disburse_time')].dt.year

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


In [16]:
temp_df.groupby('differenza').size()

differenza
0    414618
1     64219
dtype: int64

In [17]:
temp_df = temp_df[['planned_expiration_time', 'disburse_time', 'differenza', 'loan_amount', 'duration']]

In [18]:
temp_df

,planned_expiration_time,disburse_time,differenza,loan_amount,duration
3,2014-02-21 03:10:02,2014-01-17 08:00:00,0,625.0,34 days 19:10:02
9,2014-03-19 13:40:02,2014-01-10 08:00:00,0,1825.0,68 days 05:40:02
11,2014-02-20 18:30:01,2014-01-15 08:00:00,0,950.0,36 days 10:30:01
13,2014-03-26 22:24:34,2013-12-26 08:00:00,1,300.0,90 days 14:24:34
18,2015-11-03 18:40:05,2015-09-18 07:00:00,0,375.0,46 days 11:40:05
...,...,...,...,...,...
1419599,2015-12-30 20:20:04,2015-11-16 08:00:00,0,5475.0,44 days 12:20:04
1419600,2015-12-30 19:50:04,2015-11-20 08:00:00,0,500.0,40 days 11:50:04
1419601,2015-12-31 16:30:11,2015-11-19 08:00:00,0,3000.0,42 days 08:30:11
1419603,2016-01-02 16:40:07,2015-11-24 08:00:00,1,300.0,39 days 08:40:07


In [19]:
temp_df.duration = temp_df.duration.dt.days

In [20]:
def func(k):
        x = round(( k.loan_amount * k.duration)/(31+365+30),2)
        y = k['planned_expiration_time'].year
        return [y,x]

In [22]:
temp_df1 = temp_df.loc[temp_df.differenza == 0]

In [23]:
listx = []

In [24]:
listx = temp_df1.apply(lambda x : func(x), axis=1)

In [25]:
d1 = {'tot' : listx}

In [26]:
df2 = pd.DataFrame(d1)

In [27]:
df3 = df2.tot.apply(pd.Series)
df3.columns = ['year', 'total']

In [29]:
def func2(k):
        lim_inf = datetime.datetime(k.planned_expiration_time.year, 1,1)
        z = k.planned_expiration_time - lim_inf
        z = z.days
        x = round(( k.loan_amount * z)/(31+365+30),2) 
        y = k.planned_expiration_time.year
        lim_sup = datetime.datetime(k.disburse_time.year, 12, 31)
        f = lim_sup - k.disburse_time
        f = f.days
        u = round(( k.loan_amount * f)/(31+365+30),2) 
        v = k.disburse_time.year
        return [[y,x],[v,u]]

In [30]:
temp_df1 = temp_df.loc[temp_df.differenza == 1]

In [31]:
listy = []
listy = temp_df1.apply(lambda x : func2(x), axis=1)

In [32]:
listy

13            [[2014, 59.15], [2013, 2.82]]
71            [[2015, 8.98], [2014, 28.52]]
74           [[2017, 22.12], [2016, 71.48]]
75         [[2017, 237.68], [2016, 202.46]]
79          [[2014, 100.65], [2013, 22.59]]
                         ...               
1419576        [[2016, 0.0], [2015, 52.11]]
1419586     [[2016, 34.51], [2015, 540.61]]
1419589        [[2016, 0.59], [2015, 13.5]]
1419603        [[2016, 0.7], [2015, 25.35]]
1419604     [[2016, 11.38], [2015, 267.55]]
Length: 64219, dtype: object

In [33]:
d1 = {'tot' : listy}

In [34]:
df2 = pd.DataFrame(d1)

In [35]:
df4 = df2.tot.apply(pd.Series)
df4.columns = ['data1', 'data2']
df2 = df4['data1']
df4 = df4['data2']

In [36]:
d1 = {'tot' : df2}
df2 = pd.DataFrame(d1)
df5 = df2.tot.apply(pd.Series)
df5.columns = ['year', 'total']

In [37]:
df5

,year,total
13,2014.0,59.15
71,2015.0,8.98
74,2017.0,22.12
75,2017.0,237.68
79,2014.0,100.65
...,...,...
1419576,2016.0,0.00
1419586,2016.0,34.51
1419589,2016.0,0.59
1419603,2016.0,0.70


In [38]:
d1 = {'tot' : df4}
df2 = pd.DataFrame(d1)
df6 = df2.tot.apply(pd.Series)
df6.columns = ['year', 'total']

In [39]:
df6

,year,total
13,2013.0,2.82
71,2014.0,28.52
74,2016.0,71.48
75,2016.0,202.46
79,2013.0,22.59
...,...,...
1419576,2015.0,52.11
1419586,2015.0,540.61
1419589,2015.0,13.50
1419603,2015.0,25.35


In [40]:
df3 = df3.append(df5, ignore_index=True)
df3 = df3.append(df6, ignore_index=True)

In [43]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [52]:
df3

,year,total
0,2014,49.88000
1,2014,291.31000
2,2014,80.28000
3,2015,40.49000
4,2015,28.52000
...,...,...
543051,2015,52.11000
543052,2015,540.61000
543053,2015,13.50000
543054,2015,25.35000


In [48]:
df3.year = df3.year.astype('int')

In [51]:
df3.groupby('year').sum()

,total
year,
2011,29.93000
2012,74189.31000
2013,1445432.27000
2014,8922897.42000
2015,10741044.62000
2016,11705381.56000
2017,13531385.41000
2018,231284.36000


In [ ]:
def func(k,df1):
        f = k.loc[k.differenza == 0]
        x = round(( f.loan_amount * f.duration)/(31+365+30),2)
        y = f['planned_expiration_time'].dt.year
        df1 = df1.append({'Year' : y, 'Total' : x} , ignore_index=True)
        f = k.loc[k.differenza == 1]
        lim_inf = datetime.datetime(f.planned_expiration_time.dt.year, 1,1)
        z = f.planned_expiration_time - lim_inf
        z = z.dt.days
        x = ( f.loan_amount * z)/(31+365+30) 
        y = f.planned_expiration_time.dt.year
        df1 = df1.append({'Year' : y, 'Total' : x} , ignore_index=True)
        lim_sup = datetime.datetime(f.disburse_time.year, 12, 31)
        z = lim_sup - f.disburse_time
        z = z.days
        x = ( f.loan_amount * z)/(31+365+30) 
        y = f.disburse_time.year
        df1 = df1.append({'Year' : y, 'Total' : x} , ignore_index=True)    
        return 0

x = round(( temp_df1.loan_amount * temp_df1.duration)/(31+365+30),2)
y = temp_df1['planned_expiration_time'].dt.year
temp_df_years.append({'Year' : y, 'Total' : x} , ignore_index=True)

In [ ]:
def func(k):
    if k[2]==0:
        x = round(( k[3] * k[4])/(31+365+30),2)
        y = k[0].year
        temp_df_years = temp_df_years.append({'Year' : int(y), 'Total' : x} , ignore_index=True)
    else:
        lim_inf = datetime.datetime(k[0].year, 1,1)
        z = k[0] - lim_inf
        z = z.days
        x = ( k[3] * z)/(31+365+30) 
        y = k[0].year
        temp_df_years = temp_df_years.append({'Year' : int(y), 'Total' : x} , ignore_index=True)
        lim_sup = datetime.datetime(k[1].year, 12, 31)
        z = lim_sup - k[1]
        z = z.days
        x = ( k[3] * z)/(31+365+30) 
        y = k[1].year
        temp_df_years = temp_df_years.append({'Year' : int(y), 'Total' : x} , ignore_index=True)

In [ ]:
i=0
while i!=478837:
    if temp_np[i][2]==0:
        x = round(( temp_np[i][3] * temp_np[i][4])/(31+365+30),2)
        y = temp_np[i][0].year
        temp_df_years = temp_df_years.append({'Year' : int(y), 'Total' : x} , ignore_index=True)
    else:
        lim_inf = datetime.datetime(temp_np[i][0].year, 1,1)
        z = temp_np[i][0] - lim_inf
        z = z.days
        x = ( temp_np[i][3] * z)/(31+365+30) 
        y = temp_np[i][0].year
        temp_df_years = temp_df_years.append({'Year' : int(y), 'Total' : x} , ignore_index=True)
        lim_sup = datetime.datetime(temp_np[i][1].year, 12, 31)
        z = lim_sup - temp_np[i][1]
        z = z.days
        x = ( temp_np[i][3] * z)/(31+365+30) 
        y = temp_np[i][1].year
        temp_df_years = temp_df_years.append({'Year' : int(y), 'Total' : x} , ignore_index=True)
    i+=1